## Running RAJAPerf 

In this section we will be running RAJAPerf, exploring some of the properties of a set of kernels that belong to a group,  
demonstrating several techniques to display the timing Hierarchy, and creating basic bar graphs for comparisons of   
timing using compilers GCC vs Clang.

[Back to Table of Contents](./00-intro-and-contents.ipynb)

In [ ]:
%%bash
$HOME/code/RAJAPerf/build_gcc/bin/raja-perf.exe --help


In [ ]:
%%bash
# First lets run RAJAPerf, performing a dryrun and looking at the run properties, paying particular attention to the default Problem size, and Reps.
# We're also just going to specify one group of kernels to run, in this case they all fall into the Algorithm group.
# To see the full list of kernels edit the command by removing the --kernels Algorithm argument
$HOME/code/RAJAPerf/build_gcc/bin/raja-perf.exe --dryrun --kernels Algorithm

In [ ]:
%%bash
# Next lets actually generate output, but we'll pass in the argument --checkrun 1 to just run one rep of all the kernels in group Algorithm, we do this so the output is generated very quickly.
# We also specify the -sp flag to show progress timing the kernels
# Note that RAJAPerf runs a bunch of warmup kernels. These are not included in the output data files
$HOME/code/RAJAPerf/build_gcc/bin/raja-perf.exe -od $HOME/data/default_problem_size/gcc --checkrun 1 --kernels Algorithm -sp
ls $HOME/data/default_problem_size/gcc/*.cali

In [ ]:
%%bash
# Next let's inspect the files created by running RAJAPerf. In addition to the .txt and .csv files we also output a set of Caliper data .cali.
# We're going to show several techniques to display the Caliper trees (Timing Hierarchy)
# The first technique is with Caliper's own tool cali-query, we run it with -T to display tree, or you can specify --tree
# We'll focus on inspecting the timing generated by running RAJA with execution policy sequential, RAJA_Seq.cali
eval `spack env activate --sh  --dir /home/jovyan/spack_env`
eval `$HOME/spack/bin/spack load --sh caliper@master%gcc@10.4.0`
which cali-query
cali-query -T $HOME/data/default_problem_size/gcc/RAJA_Seq.cali

In [ ]:
# Next we inspect the .cali file with Caliper's own CaliperReader Python module which we installed in the previous cell
import os
import caliperreader as cr
DATA_DIR = os.getenv('HOME')+"/data/default_problem_size/gcc"
os.chdir(DATA_DIR)
r = cr.CaliperReader()
r.read("RAJA_Seq.cali")
metric = 'avg#inclusive#sum#time.duration'
for rec in r.records:
    path = rec['path'] if 'path' in rec else 'UNKNOWN'
    time = rec[metric] if metric in rec else '0'
    if not 'UNKNOWN' in path:
        if (isinstance(path, list)):
            path = "/".join(path)
        print("{0}: {1}".format(path, time))

In [ ]:
# Finally we'll inspect Caliper trees using the Hatchet Python module which we install in the previous cell
import hatchet as ht
DATA_DIR = os.getenv('HOME')+"/data/default_problem_size/gcc"
os.chdir(DATA_DIR)
gf1 = ht.GraphFrame.from_caliperreader("RAJA_Seq.cali")
print(gf1.tree())